# Battle of the Neighborhoods
## Toronto, Canada

## Opening a Pizza Shop in DownTown Toronto

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 

import requests # library to handle requests
# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          84 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0     conda-forge

The following packages will be UPDATED:

    geopy:         1.11.0-py36_0 conda-forge --> 1.18.1-py_0 conda-forge


geopy-1.18.1         | 51 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executi

In [107]:
#read dataframe from week 3
a= pd.read_csv("190118data.csv", index_col=0)
df = a[a['Borough'] == 'Scarborough'].reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811650,-79.195561
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175299
3,M1G,Scarborough,Woburn,43.768216,-79.217610
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440


In [96]:
#map Toronto
address = 'Scarborough,Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [112]:
LIMIT = 100
radius = 700
CLIENT_ID = 'SXP34AYP1ZIYOEIU2OICLU0O1XMFOIW5KA23HSWWDZMJB5WU' # your Foursquare ID
CLIENT_SECRET = 'LNNZZRZK1YAU1CQGCMGVGVK4MVEN2XT34YC35ZIIX5MXUFHN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

In [113]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c47413c4434b947c1ce237a'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Agincourt',
  'headerFullLocation': 'Agincourt, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 31,
  'suggestedBounds': {'ne': {'lat': 43.80024000630001,
    'lng': -79.25926451171674},
   'sw': {'lat': 43.78763999369999, 'lng': -79.27668748828327}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '58388814809a776da0e00646',
       'name': 'Grandeur Palace 華丽宮 (Grandeur Palace 華麗宮)',
       'location': {'address': '2301 Brimley Rd, Unit 128',
        'crossStreet': 'at Huntingwood Dr',
        'lat': 43.79788486462263,
        'lng': -79.270584

In [76]:
# function to repeat the exploring process to all the neighborhoods in Scarborough
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [77]:
# Use category id 4bf58dd8d48988d1ca941735 to only get the Pizza place
toronto_pizza = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'],
                               radius=1000, categoryIds='4bf58dd8d48988d1ca941735')
toronto_pizza.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.811650,-79.195561,Joe's Pizza,43.818744,-79.204311,Pizza Place
1,"Guildwood, Morningside, West Hill",43.765690,-79.175299,A1 Pizza,43.762176,-79.183076,Pizza Place
2,"Guildwood, Morningside, West Hill",43.765690,-79.175299,Little Caesars Pizza,43.769046,-79.184386,Pizza Place
3,Woburn,43.768216,-79.217610,Dominos Pizza,43.767048,-79.227603,Pizza Place
4,Woburn,43.768216,-79.217610,Panzerotto Pizza,43.761887,-79.225413,Pizza Place


In [78]:
toronto_pizza.shape

(38, 7)

In [79]:
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

map_toronto_pizza = folium.Map(location=[latitude, longitude], zoom_start=11)
addToMap(toronto_pizza, 'red', map_toronto_pizza)
map_toronto_pizza

In [80]:
toronto_venues_highschools = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d13d941735')
toronto_venues_highschools.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Guildwood, Morningside, West Hill",43.765690,-79.175299,West Hill Public School,43.772648,-79.187513,High School
1,Scarborough Village,43.743085,-79.232172,Clinton High School Track,43.739128,-79.237679,High School
2,"East Birchmount Park, Ionview, Kennedy Park",43.726260,-79.263670,South East Year Round Alternative Centre (S.E....,43.725370,-79.259367,High School
3,"Clairlea, Golden Mile, Oakridge",43.713213,-79.284910,SATEC @ W.A. Porter,43.716171,-79.287442,High School
4,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976,R.H. King Academy,43.722168,-79.237557,High School


In [81]:
toronto_venues_highschools.shape

(21, 7)

In [82]:
map_toronto_highschools = folium.Map(location=[latitude, longitude], zoom_start=11)
addToMap(toronto_venues_highschools, 'green', map_toronto_highschools)
map_toronto_highschools

In [128]:
toronto_venues_uni = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d1ae941735')
toronto_venues_uni.head()

https://api.foursquare.com/v2/venues/search?&client_id=SXP34AYP1ZIYOEIU2OICLU0O1XMFOIW5KA23HSWWDZMJB5WU&client_secret=LNNZZRZK1YAU1CQGCMGVGVK4MVEN2XT34YC35ZIIX5MXUFHN&v=20180605&ll=43.83421500000003,-79.21670085099998&radius=1000&limit=100&categoryId=4bf58dd8d48988d1ae941735
{'meta': {'code': 200, 'requestId': '5c4743719fb6b726b706a5f3'}, 'response': {'venues': []}}
[]
Empty DataFrame
Columns: []
Index: []


""


In [129]:
toronto_venues_uni.shape

(0, 0)

In [116]:
toronto_venues_office = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d124941735')
toronto_venues_office.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.81165,-79.195561,Tower Litho,43.809666,-79.199012,Office
1,"Rouge, Malvern",43.81165,-79.195561,Allied Technical Services,43.807999,-79.198802,Office
2,"Rouge, Malvern",43.81165,-79.195561,Imminent Concepts,43.804597,-79.199744,Coworking Space
3,"Rouge, Malvern",43.81165,-79.195561,Next Century Publishing Canada,43.804905,-79.200428,Tech Startup
4,"Rouge, Malvern",43.81165,-79.195561,Canadian feed screw,43.806616,-79.199408,Office


In [86]:
toronto_venues_office.shape

(152, 7)

In [87]:
map_toronto_office = folium.Map(location=[latitude, longitude], zoom_start=11)
addToMap(toronto_venues_office, 'fuchsia', map_toronto_office)
map_toronto_office

In [88]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [89]:
df_data = df.copy()
addColumn(df_data, 'Pizza', toronto_pizza)
addColumn(df_data, 'High Schools', toronto_venues_highschools)
addColumn(df_data, 'Offices', toronto_venues_office)
df_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Pizza,High Schools,Offices
0,M1B,Scarborough,"Rouge, Malvern",43.811650,-79.195561,1.0,0.0,8.0
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701,0.0,0.0,4.0
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175299,2.0,1.0,11.0
3,M1G,Scarborough,Woburn,43.768216,-79.217610,5.0,0.0,4.0
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440,3.0,0.0,10.0
5,M1J,Scarborough,Scarborough Village,43.743085,-79.232172,3.0,1.0,0.0
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726260,-79.263670,3.0,1.0,16.0
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713213,-79.284910,2.0,1.0,7.0
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976,2.0,2.0,6.0
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696690,-79.260069,1.0,1.0,4.0


In [90]:
# negative weight, because Luigi wants to open a pizzeria 
#and thus wants to avoid concurrence as much as possible
weight_pizza = -1

# positive weight, because high school students are good customers
weight_schools = 1

# positive weight because employees are even better customers
weight_offices = 2

In [91]:
df_weighted = df_data[['Neighborhood']].copy()

In [92]:
df_weighted['Score'] = df_data['Pizza'] * weight_pizza + df_data['High Schools'] * weight_schools + df_data['Offices'] * weight_offices
df_weighted = df_weighted.sort_values(by=['Score'], ascending=False)
df_weighted

,Neighborhood,Score
12,Agincourt,39.0
15,"L'Amoreaux West, Steeles West",36.0
10,"Dorset Park, Scarborough Town Centre, Wexford ...",35.0
6,"East Birchmount Park, Ionview, Kennedy Park",30.0
13,"Clarks Corners, Sullivan, Tam O'Shanter",28.0
2,"Guildwood, Morningside, West Hill",21.0
4,Cedarbrae,17.0
0,"Rouge, Malvern",15.0
7,"Clairlea, Golden Mile, Oakridge",13.0
11,"Maryvale, Wexford",13.0


In [104]:
pizza_win = df[df['Neighborhood'] == 'Agincourt']

pizza_win

,PostalCode,Borough,Neighborhood,Latitude,Longitude
12,M1S,Scarborough,Agincourt,43.79394,-79.267976


In [105]:
latitude = 43.793940
longitude = -79.267976

# create map of Toronto using latitude and longitude values

map_pizza_result = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map

for lat, lng, local in zip(pizza_win['Latitude'], pizza_win['Longitude'], pizza_win['Neighborhood']):
    label = '{}'.format(local)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_pizza_result) 

addToMap(toronto_pizza[toronto_pizza['Neighborhood'] == 'Agincourt'], 'red', map_pizza_result)
addToMap(toronto_venues_highschools[toronto_venues_highschools['Neighborhood'] == 'Agincourt'], 'green', map_pizza_result)
addToMap(toronto_venues_office[toronto_venues_office['Neighborhood'] == 'Agincourt'], 'fuchsia', map_pizza_result)

map_pizza_result

In [95]:
### Agincourt is the best option to open a pizzeria